In [129]:
import pandas as pd
import numpy as np
import cenpy
import os

In [130]:
cen_vars = {
        'B01003_001E': 'population', # total population
        'B06011_001E': 'income', # median household income in USD
        'B17020_002E': 'poverty_total', # number of people (incl. children) living below poverty line
        'B23025_003E': 'labor_force', # number of people who could work
        'B23025_005E': 'unemployed', # number of people in labor force but unemployed
        'B25077_001E': 'house_value', # median value of owner-occupied housing unit
        'B25064_001E': 'rent', # median rent of occupied units
       }

In [131]:
cen = {} 

for year in range(2014, 2022):
    # connect to census api
    api_database = f'ACSDT5Y{year}'
    api_conn = cenpy.remote.APIConnection(api_database)
    g_unit = 'tract'
    g_filter = {'state':'08'} # Colorado
    cen[year] = api_conn.query(cen_vars, geo_unit = g_unit, geo_filter = g_filter)

    # rename columns
    new_cols = list(cen_vars.values())
    new_cols.extend(['state_code', 'county_code', 'tract'])
    cen[year].columns = new_cols

    # calculate unemployment
    cen[year]['unemployment'] = (cen[year]['unemployed'].astype(int) / cen[year]['labor_force'].astype(int)).round(2)

    # calculate poverty rate
    cen[year]['poverty'] = (cen[year]['poverty_total'].astype(int) / cen[year]['population'].astype(int)).round(2)
    
    # add full tract number
    cen[year]['tract'] = cen[year]['state_code'] + cen[year]['county_code'] + cen[year]['tract']
    
    # drop extraneous cols
    cen[year] = cen[year].drop(['poverty_total', 'labor_force', 'unemployed', 'state_code', 'county_code'], axis = 1)
    
    # write CSV for year
    full_path = os.path.join(os.path.abspath(os.path.dirname('__file__'))) # get file location
    cen[year].to_csv(full_path + f'/census_data/co_{year}.csv')
